In this example notebook, we perform numerical simulations of DBQA for the XXZ model with a periodic boundary condition, whose hamiltonian is given by:

$$
H_0 = \sum _{k=0}^L \left( X_{k} X_{k + 1} + Y_{k} Y_{k + 1} + \delta Z_{k}Z_{k + 1} \right)
$$

In [50]:
from qibo.backends import construct_backend
from qibo import hamiltonians, gates, models, Circuit, set_backend
import matplotlib.pyplot as plt
import numpy as np
import time
import networkx as nx

In [51]:
def prepare_singlet_state(nqubits, M=None, qc=None):
    """
    Prepare tensor product of singlet states
    """
    if qc is None:
        qc = Circuit(nqubits)
    def sing(a, b):
        qc.add(gates.X(a))
        qc.add(gates.H(a))
        qc.add(gates.X(b))
        qc.add(gates.CNOT(a, b))
    if M is None:
        # Count pairs up till the last qubit (for chain-like systems)
        for i in range(0, nqubits, 2):
            if i + 1 < nqubits:
                sing(i, i + 1)
    else:
        for i, j in M:
            sing(i, j)
    return qc

def exact_expectation_circ(ham, circ):
    # calculates the exact expectation of hamiltonian given a circuit in qibo
    return ham.expectation(
        ham.backend.execute_circuit(circuit=circ).state())

In [52]:
L = 10
G = nx.Graph()
G.add_edges_from([(k, (k+1)%L) for k in range(L)]) # periodic boundary condition
M = nx.maximal_matching(G)
delta = 0.5
# closed boundary condition
H = hamiltonians.XXZ(L, delta, dense=False)
singlet_qc = prepare_singlet_state(L)
vqe_file = f'results/circuit_qasm/cobyla_{L}q_1l_XXZ/vqe_circ.qasm'
# with open(vqe_file, 'r') as f:
#     vqe_circ = Circuit.from_qasm(f.read())

# print(HVA_qc.draw())
psi0 = H.backend.zero_state(L)
print("Initial energy:", H.expectation(psi0))
print("Singlet energy:", exact_expectation_circ(H, singlet_qc))
# print('VQE energy:', exact_expectation_circ(H, vqe_circ))

Initial energy: 5.0
Singlet energy: -12.499999999999988


In [53]:
def XXZ_HVA_ansatz(G, delta, nlayers=1, parameters=None):
    # 2 parameters per layer: one for each of the even and odd pairs
    if parameters is None:
        parameters = [1] * (nlayers * 2)
    if len(parameters)  != nlayers * 2:
        raise ValueError(f"Expected {nlayers * 2} parameters, got {len(parameters)}")
    
    # initialize the circuit
    nqubits = G.number_of_nodes()
    M = nx.maximal_matching(G)
    qc = Circuit(nqubits)
    # prepare the singlet state
    qc = prepare_singlet_state(nqubits, M=M, qc=qc)

    for i in range(nlayers):
        # apply the even starting pairs
        for p, q in M:
            # XX
            # qc.add(gates.RXX(p, q, parameters[i * 2]))
            qc.add(gates.H(p)), qc.add(gates.H(q))
            qc.add(gates.RZZ(p, q, parameters[i * 2]))
            qc.add(gates.H(p)), qc.add(gates.H(q))
            # YY
            # qc.add(gates.RYY(p, q, parameters[i * 2]))
            qc.add(gates.SDG(p)), qc.add(gates.SDG(q))
            qc.add(gates.H(p)), qc.add(gates.H(q))
            qc.add(gates.RZZ(p, q, parameters[i * 2]))
            qc.add(gates.H(p)), qc.add(gates.H(q))
            qc.add(gates.S(p)), qc.add(gates.S(q))
            # ZZ
            qc.add(gates.RZZ(p, q, parameters[i * 2]*delta))
        # apply the odd starting pairs
        for p, q in set(G.edges()-M):
            # XX
            # qc.add(gates.RXX(p, q, parameters[i * 2]))
            qc.add(gates.H(p)), qc.add(gates.H(q))
            qc.add(gates.RZZ(p, q, parameters[i * 2+1]))
            qc.add(gates.H(p)), qc.add(gates.H(q))
            # YY
            # qc.add(gates.RYY(p, q, parameters[i * 2]))
            qc.add(gates.SDG(p)), qc.add(gates.SDG(q))
            qc.add(gates.H(p)), qc.add(gates.H(q))
            qc.add(gates.RZZ(p, q, parameters[i * 2+1]))
            qc.add(gates.H(p)), qc.add(gates.H(q))
            qc.add(gates.S(p)), qc.add(gates.S(q))
            # ZZ
            qc.add(gates.RZZ(p, q, parameters[i * 2+1]*delta))              
    return qc

In [54]:
# optimize HVA
nlayers = 2
objective = lambda params: exact_expectation_circ(H, XXZ_HVA_ansatz(G, delta, nlayers=nlayers, parameters=params))
initial_params = [0.25] * (nlayers * 2)
print('Initial loss:', objective(initial_params))

Initial loss: -10.917862977206433


In [55]:
from scipy.optimize import minimize
max_iter = 1000
result = minimize(
    objective,
    initial_params,
    method="COBYLA",
    options={"disp": True, "maxiter": max_iter},
    tol=1e-2,
)

print(result.fun)
print(result.x)

Return from COBYLA because the trust region radius reaches its lower bound.
Number of function values = 40   Least value of F = -14.358086266630005
The corresponding X is:
[ 1.24921867  0.34121785  0.41495117 -0.12760757]

-14.358086266630005
[ 1.24921867  0.34121785  0.41495117 -0.12760757]


In [56]:
hva_circ = XXZ_HVA_ansatz(G, delta, nlayers=nlayers, parameters=result.x)

In [57]:
print("HVA energy:", exact_expectation_circ(H, hva_circ))
# print("VQE energy:", exact_expectation_circ(H, vqe_circ))
print("Singlet energy:", exact_expectation_circ(H, singlet_qc))

HVA energy: -14.358086266630005
Singlet energy: -12.499999999999988


In [58]:
import os
hva_circ_qasm = models.Circuit.to_qasm(hva_circ)
folder_path = f"results/circuit_qasm/cobyla_{L}q_{nlayers}l_XXZ/"
file_path = folder_path + f"hva_circ.qasm"
os.makedirs(folder_path, exist_ok=True)
with open(file_path, 'w') as f:
    f.write(hva_circ_qasm)

In [61]:
import pytket.qasm
hva_circ_pytket = pytket.qasm.circuit_from_qasm(file_path)
psi_hva = hva_circ_pytket.get_unitary() @ psi0
print(np.vdot(psi_hva, H.matrix @ psi_hva).real)

-14.358086266630751
